In [1]:
import numpy as np
import csv
from datetime import datetime
from ansys.mapdl.core import launch_mapdl

# ----- Import your existing geometry functions here -----
from run_analysis import (
    model_rectangular_beam,
    model_spherical_beam,
    init_mapdl,
)

output_file = "ml_dataset.csv"
header_written = False

def run_case(mapdl, shape_name, result, params, writer):
    row = [
        datetime.now().isoformat(timespec="seconds"),
        shape_name,
        *params,
        result
    ]
    writer.writerow(row)

with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "timestamp", "shape",
        "param1", "param2", "param3", "param4",  # depends on shape
        "max_deformation"
    ])

    # Launch once, reuse MAPDL instance
    mapdl = launch_mapdl(port=None, override=True)
    mapdl.clear()
    mapdl.prep7()
    mapdl.units("SI")
    mapdl.et(1, "SOLID185")
    mapdl.mp("EX", 1, 210e9)
    mapdl.mp("PRXY", 1, 0.3)

    # --- Rectangular beam cases ---
    for _ in range(30):
        length = np.random.uniform(0.2, 0.6)
        width = np.random.uniform(0.02, 0.08)
        height = np.random.uniform(0.02, 0.08)
        pressure = np.random.uniform(1e6, 8e6)
        result = model_rectangular_beam(mapdl, length, width, height, pressure)
        run_case(mapdl, "rect", result, [length, width, height, pressure], writer)

    # --- Sphere (cube-approximation) cases ---
    for _ in range(30):
        radius = np.random.uniform(0.05, 0.15)
        pressure = np.random.uniform(1e6, 8e6)
        result = model_spherical_beam(mapdl, radius, pressure)
        run_case(mapdl, "sphere", result, [radius, None, None, pressure], writer)

    mapdl.exit()

print(f"✅ Dataset generated in: {output_file}")



ModuleNotFoundError: No module named 'run_analysis'